In [17]:
import numpy as np
import pandas as pd
import libpysal.weights as lpw
from scipy.linalg import lu
from scipy.optimize import minimize

# 1. Generate location data $X$

In [18]:
# function that makes coordincates of data points
def make_coords(N_points, xmu, xsd, ymu, ysd):  
    x_coords = np.random.normal(xmu, xsd, size=N_points)
    y_coords = np.random.normal(ymu, ysd, size=N_points)
    coords = np.column_stack((x_coords, y_coords))
    return coords

* 수정) For문보다 효율적으로 Distance 구하는 방법 찾아보기

In [19]:
# function that calculates distances between coordinates
def distances(coords):
    N = len(coords)
        
    # Initialize an empty distances matrix
    distances = np.zeros((N, N))

    # Compute the euclidean distance between pair of coords
    for i in range(N):
        for j in range(i+1, N):
            distances[i,j] = distances[j,i] = np.linalg.norm(coords[i] - coords[j])       

    return distances

In [20]:
np.random.seed(24)

In [21]:
# set parameters
p = 10 # dimensionaltiy
paramTrue = [1, 1/2] # parameters 

In [22]:
# get coordinates
coord = [0, 1, 0, 1] # xmu, xsd, ymu, ysd
X = make_coords(p, *coord)
X

array([[ 1.32921217,  0.6788048 ],
       [-0.77003345,  1.88927273],
       [-0.31628036,  0.9615384 ],
       [-0.99081039,  0.1040112 ],
       [-1.07081626, -0.48116532],
       [-1.43871328,  0.85022853],
       [ 0.56441685,  1.45342467],
       [ 0.29572189,  1.05773744],
       [-1.62640423,  0.16556161],
       [ 0.2195652 ,  0.51501838]])

In [23]:
# get distances
d = distances(X)

# 2. Generate Covariance Matrix $\Sigma$
$$ \Sigma = Cov(Y_i, Y_j|X) = \theta_1 \exp(-\frac{1}{\theta_2} d_{ij}) \\ d_{ij} = ||X_i - X_j||$$

In [24]:
def Sigma(param, dis):
  theta1, theta2 = param[0], param[1]
  cov = theta1*np.exp((-1/theta2)*dis) # dis is a square matrix
  return cov

In [25]:
# get SigmaTrue
SigmaTrue = Sigma(paramTrue, d)

# 3. Generate data $Y$
$$ Y = \Sigma^{1/2} \cdot U \\ U \sim N(0, 1) \\ Y \sim N(0, \Sigma) $$

In [26]:
SigmaHalf = np.linalg.cholesky(SigmaTrue)
U = np.random.normal(0, 1, p).reshape(p, 1)
Y = np.dot(SigmaHalf, U)
Y.shape # dimensionality of Y


(10, 1)

# 4. Parameter Estimation

$$ logL(\vec{\theta};\vec{Y}) = -\frac{1}{2} \log ( |\Sigma(\theta)|) - \frac{1}{2} \vec{Y}^T (\Sigma(\theta))^{-1} \vec{Y} $$

* Constraints regarding $ \Sigma(\theta) $
    1. When calculating the determinant of $\Sigma(\theta)$, we need to consider the following constraints
    * Symmetric: As distance matrix is symmetric, $\Sigma(\theta)$ is also symmetric
    * Positive Definite
        * If it's pd, we can use choleskey factorization prior to the calculation of log determinant.
        * If it's not pd, we can use LU factorization. 
        * (This helps preventing over/underflow when calculating determinant of large matrix)
    2. When calculating the inverse of $\Sigma(\theta)$, we need to consider if it's invertible or not.

In [27]:
# check pd
def check_pd(Sigma):
    if np.linalg.eig(Sigma)[0].min() > 0:
        return True
    else:
        return False

In [28]:
# check invertible
def check_invertible(matrix):
    try:
        inverse = np.linalg.inv(matrix)
        return True
    except np.linalg.LinAlgError:
        return False

## 4-1. Likelihood function

In [29]:
# define log likelihood function
def logLikelihood(param, Y):
    
    p = len(Y)
    sig = Sigma(param, d)
    chol = check_pd(sig)
    inv = check_invertible(sig)
    
    if chol: # pd check
        # choleskey factorization; positive definite and symmetric
        log_det_Sigma = 2 * np.sum(np.log(np.diag(np.linalg.cholesky(sig))))
    else: 
        # LU factorization; square
        P, L, U = lu(sig)
        du = np.diag(U)
        c = np.linalg.det(P) * np.prod(np.sign(du))
        log_det_Sigma = np.log(c) + np.sum(np.log(np.abs(du)))

    
    if inv:
        inv_Sigma = np.linalg.inv(sig)
    else:   
        inv_Sigma = np.linalg.pinv(sig) # pseudo inverse
    
        
    return 0.5 * Y.T @ inv_Sigma @ Y + 0.5 * log_det_Sigma

In [30]:
# # define log likelihood function
# def logLikelihood(param, Y):
    
#     p = len(Y)
#     sig = Sigma(param, d)
#     inv_Sigma = np.linalg.inv(sig)
    
#     log_det_Sigma = 2 * np.sum(np.log(np.diag(np.linalg.cholesky(sig)))) # reduce computation complexity

#     # det_Sigma = np.linalg.det(sig)
#     # log_det_Sigma = np.log(det_Sigma)
        
#     return 0.5 * Y.T @ inv_Sigma @ Y + 0.5 * log_det_Sigma


In [31]:
param0 = paramTrue

# # Define the constraint function
# def constraint1(param, Y):
#     p = len(Y)
#     sig = Sigma(param, p)
#     eigenvalues = np.linalg.eig(sig)[0]
#     return eigenvalues.min() - 1e-10 # Constraints: eigenvalues > 0

                
# Define the optimization problem
problem = {
    'fun': logLikelihood,         # Objective function: minimize the minimum eigenvalue
    'x0': paramTrue,            # Initial guess for the parameters
    'args': (Y,),             # Additional arguments for the objective function 
    # 'constraints': {'type': 'eq', 'fun': constraint1, 'args':(Y,)},  # Constraint function: eigenvalues >= 0 ineq
    'method': 'BFGS'        
}

result = minimize(**problem)

estimated_params = result.x

In [32]:
print("MLE = ", estimated_params)

array([1.03933337, 0.8866519 ])